> Code for discrete VKH limit

In [ ]:
j_lvector = np.linspace (minj_l, maxj_l, num = testpointsj_l)
j_gvector = np.linspace (minj_g, maxj_g, num = testpointsj_g)
j_lmapstability_disc = np.zeros ((testpointsbeta,testpointsj_l)) #(rows,columns)

k = 0
for beta in betavector:
    print ("  >>> β = ", np.rad2deg(beta), "[deg]")
    j = 0 #for each element of j2vector
    for j_g in j_gvector: #iterative process for j2
        i = 0 #for each element of j1vector
        for j_l in j_lvector: #iterative process for j1
            var4 = var4_0
            var1eq = brenth (equilibrium1, lima, limb)
            
            # parameters used in boundary conditions
            var1_0 = var1eq
            var2_0 = var2 (var1eq)
            var3_0 = var3 (var1eq)
            ref = np.array([var1_0, var2_0, var3_0, var4_0])
            
            # Import notebooks
            %run ./codes/11_reference_conditions.ipynb
            %run ./codes/18_fem_discretization.ipynb
            %run ./codes/19_initial_boundary_conditions.ipynb
            %run ./codes/20_variational_form.ipynb
            # %run ./codes/21_solvers.ipynb

            # ===============================================================
            # FULLY-DISCRETIZED
            # ===============================================================
            if any ([system == 1, system == 2]):   
                # L2 norm for each time step
                L2norm_variable1 = []
                L2norm_variable2 = []
                L2norm_variable3 = []
                L2norm_variable4 = []

                # Time vector for L2 norm computation
                timevector = np.linspace (0, T, num_steps)

                # Step in time
                t = 0
                while (t < T):
                    # Condition for Dirichlet boundary conditions
                    if IBVP == 2:
                        variable1_dirichlet.timedirichlet = t
                        variable2_dirichlet.timedirichlet = t
                        variable3_dirichlet.timedirichlet = t
                        variable4_dirichlet.timedirichlet = t

                    # Initial conditions    
                    if any ([time_method == 1 , time_method == 3]):
                        if t == 0:
                            (variable1_n, variable2_n, variable3_n, variable4_n) = variable_n.split (deepcopy = True)
                            # Compute  nodal values (initial)
                            n_val1_n = np.array (variable1_n.vector ())
                            n_val2_n = np.array (variable2_n.vector ())
                            n_val3_n = np.array (variable3_n.vector ())
                            n_val4_n = np.array (variable4_n.vector ())

                            nodal_variable1_n = n_val1_n [::-1]
                            nodal_variable2_n = n_val2_n [::-1]
                            nodal_variable3_n = n_val3_n [::-1]
                            nodal_variable4_n = n_val4_n [::-1]

                            # print ("n", nodal_variable1_n)

                            nodes_variable1_n = len (nodal_variable1_n)
                            nodes_variable2_n = len (nodal_variable2_n)
                            nodes_variable3_n = len (nodal_variable3_n)
                            nodes_variable4_n = len (nodal_variable4_n)

                            # print ("dof subspace variable1 =", nodes_variable1_n)
                            # print ("dof subspace variable2 =", nodes_variable2_n)
                            # print ("dof subspace variable3 =", nodes_variable3_n)
                            # print ("dof subspace variable4 =", nodes_variable4_n)

                            # Compute vertex values (initial step)
                            v_variable1_n = variable1_n.compute_vertex_values (mesh)
                            v_variable2_n = variable2_n.compute_vertex_values (mesh)
                            v_variable3_n = variable3_n.compute_vertex_values (mesh)
                            v_variable4_n = variable4_n.compute_vertex_values (mesh)

                            vert_variable1_n = []
                            vert_variable2_n = []
                            vert_variable3_n = []
                            vert_variable4_n = []
                            
                            for cond_vert in range (len (coordinates)):
                                    vert_variable1_n.append (v_variable1_n[cond_vert])
                                    vert_variable2_n.append (v_variable2_n[cond_vert])
                                    vert_variable3_n.append (v_variable3_n[cond_vert])
                                    vert_variable4_n.append (v_variable4_n[cond_vert])
                            
                            vert_variable1_n = np.asarray (vert_variable1_n)
                            vert_variable2_n = np.asarray (vert_variable2_n)
                            vert_variable3_n = np.asarray (vert_variable3_n)
                            vert_variable4_n = np.asarray (vert_variable4_n)
                            
                            if show_data == 1:
                                print ("variable1_n = ", vert_variable1_n)

                        # Time stepping
                        t += dt
                        
                        # Print progress
                        if show_data == 1:
                            print ("Iteration :", int (round (t/dt)), "of", num_steps)
                            print ("Time      :", t, "s")
                        
                        # Amat
                        Am1 = ( (a0*variable1 + a1*variable1_n + a2*variable1_past)/dt*v1 )*dx 
                        Am2 = ( (a0*variable2 + a1*variable2_n + a2*variable1_past)/dt*v2 )*dx 
                        Am3 = ( (a0*variable3 + a1*variable3_n + a2*variable1_past)/dt*v3 )*dx 
                        Am4 = ( (a0*variable4 + a1*variable4_n + a2*variable1_past)/dt*v4 )*dx

                        Am = Am1 + Am2 + Am3 + Am4

                        # Variational form
                        F = visc*Cm - Bm
                        F_n = visc*Cm_n - Bm_n
                        R = Am - theta*F - (1 - theta)*F_n 

                        # Compute directional derivative about u in the direction of du (Jacobian)
                        dF = derivative (R, variable, dvariable)

                        # Define transient solver function
                        transientsolver = solver_linear (R, variable, bcs, dF)
                        transientsolver.solve ()

                        # Split of the vector var
                        (variable1, variable2, variable3, variable4) = variable.split (deepcopy = True)

                        # L2 norm
                        # infonorm = variable1.vector ().norm("l2")
                        # print ("Solution vector norm (0): {!r}".format (infonorm)) 
                        # L2norm_variable1.append (variable1.vector ().norm("l2"))
                        # L2norm_variable2.append (variable2.vector ().norm("l2"))   
                        # L2norm_variable3.append (variable3.vector ().norm("l2"))   
                        # L2norm_variable4.append (variable4.vector ().norm("l2"))   

                        # Nodal values (current)
                        n_val1_n1 = np.array(variable1.vector ())
                        n_val2_n1 = np.array(variable2.vector ())
                        n_val3_n1 = np.array(variable3.vector ())
                        n_val4_n1 = np.array(variable4.vector ())

                        nodal_variable1_n1 = n_val1_n1 [::-1]
                        nodal_variable2_n1 = n_val2_n1 [::-1]
                        nodal_variable3_n1 = n_val3_n1 [::-1]
                        nodal_variable4_n1 = n_val4_n1 [::-1]

                        # print ("n+1", nodal_variable1_n1)

                        # Compute vertex values (next step)
                        vert_variable1 = variable1.compute_vertex_values (mesh)
                        vert_variable2 = variable2.compute_vertex_values (mesh)
                        vert_variable3 = variable3.compute_vertex_values (mesh)
                        vert_variable4 = variable4.compute_vertex_values (mesh)

                        vert_variable1_n1 = []
                        vert_variable2_n1 = []
                        vert_variable3_n1 = []
                        vert_variable4_n1 = []

                        for cond_vert in range (len (coordinates)):
                            vert_variable1_n1.append (vert_variable1[cond_vert])
                            vert_variable2_n1.append (vert_variable2[cond_vert])
                            vert_variable3_n1.append (vert_variable3[cond_vert])
                            vert_variable4_n1.append (vert_variable4[cond_vert])
                    
                        vert_variable1_n1 = np.asarray (vert_variable1_n1)
                        vert_variable2_n1 = np.asarray (vert_variable2_n1)
                        vert_variable3_n1 = np.asarray (vert_variable3_n1)
                        vert_variable4_n1 = np.asarray (vert_variable4_n1)

                        # print ("variable1_n1 = ", vert_variable1_n1)

                        # if simulation == 'linear_simulation':
                        #     # Well-posedness check
                        #     i = 0
                        #     for variable1_test, variable2_test, variable3_test, variable4_test in zip (vert_variable1_n1, vert_variable2_n1, vert_variable3_n1, vert_variable4_n1):
                        #         Aeval = np.asarray (Amat (variable1_test, variable2_test, variable3_test, variable4_test))
                        #         Beval = np.asarray (Bmat (variable1_test, variable2_test, variable3_test, variable4_test))

                        #         m_test, _ = eig (Beval, b = Aeval, overwrite_a = True, overwrite_b = True, check_finite = True)

                        #         listreal = []
                        #         listimag = []

                        #         for cond0 in range (len (m_test)):
                        #             realpart = m_test [cond0].real                
                        #             imagpart = m_test [cond0].imag
                                    
                        #             listimag.append (imagpart)
                        #             listreal.append (realpart)
                                
                        #         if any ([ cond1 != 0 for cond1 in listimag ]) :
                        #             print ("INFO: ill-posed equation system")
                        #             break
                        #         else:
                        #             i += 1
                        #         break

                        #     # Condition for low liquid level of transition to intermittent flow
                        #     if any ([cond2 <= 0 for cond2 in vert_variable1_n1]) :
                        #         print ("INFO: gas phase dominant. Low liquid level. Liquid equations vanish")
                        #         break
                        #     elif any ([cond2 >= 1 for cond2 in vert_variable1_n1]):
                        #         print ("INFO: liquid phase dominant. Intermittent flow. Gas equations vanish")
                        #         break
                        #     else:
                        #         pass
                        #     # Condition for positive velocities and pressure
                        #     if any ([ cond3 <= 0 for cond3 in vert_variable2_n1]) :
                        #         print ("INFO: negative liquid velocity")
                        #         break
                        #     else:
                        #         pass

                        #     if any ([ cond4 <= 0 for cond4 in vert_variable3_n1]) :
                        #         print ("INFO: negative gas velocity")
                        #         break
                        #     else:
                        #         pass

                        #     if any ([ cond5 <= 0 for cond5 in vert_variable4_n1]) :
                        #         print ("INFO: negative pressure")
                        #         break
                        #     else:
                        #         pass
                            
                        #     # Reynolds check
                        #     Rel_n1 = Re_l (vert_variable1_n1, vert_variable2_n1)
                        #     Reg_n1 = Re_g (vert_variable1_n1, vert_variable3_n1, vert_variable4_n1)

                        #     if any ([ cond3 <= 1180 for cond3 in Rel_n1]):
                        #     # Rel < 1180:
                        #         print ("Liquid laminar")
                        #     elif  any ([ cond3 > 1180 for cond3 in Rel_n1]):
                        #         pass

                        #     if any ([ cond4 <= 1180 for cond4 in Reg_n1]):
                        #         print ("Gas laminar")
                        #     elif  any ([ cond4 > 1180 for cond4 in Reg_n1]):
                        #         pass

            # ===============================================================
            # CURRENT TIME BDF2
            # ===============================================================
                    if time_method == 2:
                        # Initial conditions
                        if t == 0:
                            (variable1_past, variable2_past, variable3_past, variable4_past) = variable_past.split (deepcopy = True)
                            # Compute  nodal values (initial)
                            # n_val1_past = np.array (variable1_past.vector ())
                            # n_val2_past = np.array (variable2_past.vector ())
                            # n_val3_past = np.array (variable3_past.vector ())
                            # n_val4_past = np.array (variable4_past.vector ())

                            # nodal_variable1_past = n_val1_past [::-1]
                            # nodal_variable2_past = n_val2_past [::-1]
                            # nodal_variable3_past = n_val3_past [::-1]
                            # nodal_variable4_past = n_val4_past [::-1]

                            # # print ("n", nodal_variable1_n)

                            # nodes_variable1_past = len (nodal_variable1_past)
                            # nodes_variable2_past = len (nodal_variable2_past)
                            # nodes_variable3_past = len (nodal_variable3_past)
                            # nodes_variable4_past = len (nodal_variable4_past)

                            # print ("dof subspace variable1 =", nodes_variable1_past)
                            # print ("dof subspace variable2 =", nodes_variable2_past)
                            # print ("dof subspace variable3 =", nodes_variable3_past)
                            # print ("dof subspace variable4 =", nodes_variable4_past)

                            # Compute vertex values (initial step)
                            v_variable1_past = variable1_past.compute_vertex_values (mesh)
                            v_variable2_past = variable2_past.compute_vertex_values (mesh)
                            v_variable3_past = variable3_past.compute_vertex_values (mesh)
                            v_variable4_past = variable4_past.compute_vertex_values (mesh)

                            vert_variable1_past = []
                            vert_variable2_past = []
                            vert_variable3_past = []
                            vert_variable4_past = []

                            for cond_vert in range (len (coordinates)):
                                vert_variable1_past.append (v_variable1_past[cond_vert])
                                vert_variable2_past.append (v_variable2_past[cond_vert])
                                vert_variable3_past.append (v_variable3_past[cond_vert])
                                vert_variable4_past.append (v_variable4_past[cond_vert])
                            
                            vert_variable1_past = np.asarray (vert_variable1_past)
                            vert_variable2_past = np.asarray (vert_variable2_past)
                            vert_variable3_past = np.asarray (vert_variable3_past)
                            vert_variable4_past = np.asarray (vert_variable4_past)

                            if show_data == 1:
                                print ("variable1_past = ", vert_variable1_past)

                            # # Well-posedness check
                            # i = 0
                            # for variable1_test, variable2_test, variable3_test, variable4_test in zip (vert_variable1_n, vert_variable2_n, vert_variable3_n, vert_variable4_n):
                            #     Aeval = np.asarray (Amat (variable1_test, variable2_test, variable3_test, variable4_test))
                            #     Beval = np.asarray (Bmat (variable1_test, variable2_test, variable3_test, variable4_test))

                            #     m_test, _ = eig (Beval, b = Aeval, overwrite_a = True, overwrite_b = True, check_finite = True)

                            #     listreal = []
                            #     listimag = []

                            #     for cond0 in range (len (m_test)):
                            #         realpart = m_test [cond0].real                
                            #         imagpart = m_test [cond0].imag
                                    
                            #         listimag.append (imagpart)
                            #         listreal.append (realpart)
                                
                            #     if any ([ cond1 != 0 for cond1 in listimag ]) :
                            #         print ("INFO: ill-posed equation system")
                            #         break
                            #     else:
                            #         i += 1
                            #     break

                            # # Condition for low liquid level of transition to intermittent flow
                            # if any ([cond2 <= 0 for cond2 in vert_variable1_n]) :
                            #     print ("INFO: gas phase dominant. Low liquid level. Liquid equations vanish")
                            #     break
                            # elif any ([cond2 >= 1 for cond2 in vert_variable1_n]):
                            #     print ("INFO: liquid phase dominant. Intermittent flow. Gas equations vanish")
                            #     break
                            # else:
                            #     pass
                            # # Condition for positive velocities and pressure
                            # if any ([ cond3 <= 0 for cond3 in vert_variable2_n]) :
                            #     print ("INFO: negative liquid velocity")
                            #     break
                            # else:
                            #     pass

                            # if any ([ cond4 <= 0 for cond4 in vert_variable3_n]) :
                            #     print ("INFO: negative gas velocity")
                            #     break
                            # else:
                            #     pass

                            # if any ([ cond5 <= 0 for cond5 in vert_variable4_n]) :
                            #     print ("INFO: negative pressure")
                            #     break
                            # else:
                            #     pass
                            
                            # # Reynolds check
                            # Rel_n = Re_l (vert_variable1_n, vert_variable2_n)
                            # Reg_n = Re_g (vert_variable1_n, vert_variable3_n, vert_variable4_n)

                            # if any ([ cond3 <= 1180 for cond3 in Rel_n]):
                            # # Rel < 1180:
                            #     print ("Liquid laminar")
                            # elif  any ([ cond3 > 1180 for cond3 in Rel_n]):
                            #     pass

                            # if any ([ cond4 <= 1180 for cond4 in Reg_n]):
                            #     print ("Gas laminar")
                            # elif  any ([ cond4 > 1180 for cond4 in Reg_n]):
                            #     pass

                            # Plot solution
                            # plt.figure (5, figsize = mapsize)
                            # plt.xlim (0, L)
                            # plt.ylim (0, 1)
                            # plt.grid (True, which = "both")
                            # plot (variable1_past, title = r"$\alpha_l$", rescale = False)

                            # plt.figure (6, figsize = mapsize)
                            # plt.xlim (0, L)
                            # plt.ylim (min (vert_variable2_past), max (vert_variable2_past))
                            # plt.grid (True, which = "both")
                            # plot (variable2_past, title = r"$u_l$", rescale = False)

                            # plt.figure (7, figsize = mapsize)
                            # plt.xlim (0, L)
                            # plt.grid (True, which = "both")
                            # plot (variable3_past, title = r"$u_g$", rescale = False)

                            # plt.figure (8, figsize = mapsize)
                            # plt.xlim (0, L)
                            # plt.grid (True, which = "both")
                            # plot (variable4_past, title = r"$p_i$", rescale = False)
                            
                            # # Save solution
                            # ff_variable1 << variable1_past
                            # ff_variable2 << variable2_past
                            # ff_variable3 << variable3_past
                            # ff_variable4 << variable4_past

                            # Time stepping
                            t = dt/2
                                            
                            # Parameters
                            a0 = 1
                            a1 = -1
                            a2 = 0
                            theta = 1
                            
                            # Amat
                            Am1 = ((a0*variable1_n + a1*variable1_past)/dt*v1)*dx 
                            Am2 = ((a0*variable2_n + a1*variable1_past)/dt*v2)*dx 
                            Am3 = ((a0*variable3_n + a1*variable1_past)/dt*v3)*dx 
                            Am4 = ((a0*variable4_n + a1*variable1_past)/dt*v4)*dx

                            Am = Am1 + Am2 + Am3 + Am4

                            # Variational form
                            F_n = visc*Cm_n - Bm_n
                            F_past = visc*Cm_past - Bm_past
                            R = Am - theta*F_n - (1 - theta)*F_past 

                            # Compute directional derivative about u in the direction of du (Jacobian)
                            dF = derivative (R, variable_n, dvariable)

                            # Define transient solver function
                            transientsolver = solver_linear (R, variable, bcs, dF)
                            transientsolver.solve ()

                            # Split of the vector var
                            (variable1, variable2, variable3, variable4) = variable.split (deepcopy = True)
                            # L2 norm
                            # infonorm = variable1.vector ().norm("l2")
                            # print ("Solution vector norm (0): {!r}".format (infonorm)) 
                            # L2norm_variable1.append (variable1.vector ().norm ("l2"))
                            # L2norm_variable2.append (variable2.vector ().norm ("l2"))   
                            # L2norm_variable3.append (variable3.vector ().norm ("l2"))   
                            # L2norm_variable4.append (variable4.vector ().norm ("l2"))   

                            # Nodal values (current)
                            # n_val1_n1 = np.array (variable1.vector ())
                            # n_val2_n1 = np.array (variable2.vector ())
                            # n_val3_n1 = np.array (variable3.vector ())
                            # n_val4_n1 = np.array (variable4.vector ())

                            # nodal_variable1_n1 = n_val1_n1 [::-1]
                            # nodal_variable2_n1 = n_val2_n1 [::-1]
                            # nodal_variable3_n1 = n_val3_n1 [::-1]
                            # nodal_variable4_n1 = n_val4_n1 [::-1]

                            # # print ("n+1", nodal_variable1_n1)

                            # # Compute vertex values (next step)
                            # vert_variable1 = variable1.compute_vertex_values (mesh)
                            # vert_variable2 = variable2.compute_vertex_values (mesh)
                            # vert_variable3 = variable3.compute_vertex_values (mesh)
                            # vert_variable4 = variable4.compute_vertex_values (mesh)

                            # vert_variable1_n1 = []
                            # vert_variable2_n1 = []
                            # vert_variable3_n1 = []
                            # vert_variable4_n1 = []

                            # for cond_vert in range (len (coordinates)):
                            #     vert_variable1_n1.append (vert_variable1[cond_vert])
                            #     vert_variable2_n1.append (vert_variable2[cond_vert])
                            #     vert_variable3_n1.append (vert_variable3[cond_vert])
                            #     vert_variable4_n1.append (vert_variable4[cond_vert])
                        
                            # vert_variable1_n1 = np.asarray (vert_variable1_n1)
                            # vert_variable2_n1 = np.asarray (vert_variable2_n1)
                            # vert_variable3_n1 = np.asarray (vert_variable3_n1)
                            # vert_variable4_n1 = np.asarray (vert_variable4_n1)

                            # print ("variable1_n1 = ", vert_variable1_n1)

                            # # Well-posedness check
                            # i = 0
                            # for variable1_test, variable2_test, variable3_test, variable4_test in zip (vert_variable1_n1, vert_variable2_n1, vert_variable3_n1, vert_variable4_n1):
                            #     Aeval = np.asarray (Amat (variable1_test, variable2_test, variable3_test, variable4_test))
                            #     Beval = np.asarray (Bmat (variable1_test, variable2_test, variable3_test, variable4_test))

                            #     m_test, _ = eig (Beval, b = Aeval, overwrite_a = True, overwrite_b = True, check_finite = True)

                            #     listreal = []
                            #     listimag = []

                            #     for cond0 in range (len (m_test)):
                            #         realpart = m_test [cond0].real                
                            #         imagpart = m_test [cond0].imag
                                    
                            #         listimag.append (imagpart)
                            #         listreal.append (realpart)
                                
                            #     if any ([ cond1 != 0 for cond1 in listimag ]) :
                            #         print ("INFO: ill-posed equation system")
                            #         break
                            #     else:
                            #         i += 1
                            #     break

                            # # Condition for low liquid level of transition to intermittent flow
                            # if any ([cond2 <= 0 for cond2 in vert_variable1_n1]) :
                            #     print ("INFO: gas phase dominant. Low liquid level. Liquid equations vanish")
                            #     break
                            # elif any ([cond2 >= 1 for cond2 in vert_variable1_n1]):
                            #     print ("INFO: liquid phase dominant. Intermittent flow. Gas equations vanish")
                            #     break
                            # else:
                            #     pass
                            # # Condition for positive velocities and pressure
                            # if any ([ cond3 <= 0 for cond3 in vert_variable2_n1]) :
                            #     print ("INFO: negative liquid velocity")
                            #     break
                            # else:
                            #     pass

                            # if any ([ cond4 <= 0 for cond4 in vert_variable3_n1]) :
                            #     print ("INFO: negative gas velocity")
                            #     break
                            # else:
                            #     pass

                            # if any ([ cond5 <= 0 for cond5 in vert_variable4_n1]) :
                            #     print ("INFO: negative pressure")
                            #     break
                            # else:
                            #     pass
                            
                            # # Reynolds check
                            # Rel_n1 = Re_l (vert_variable1_n1, vert_variable2_n1)
                            # Reg_n1 = Re_g (vert_variable1_n1, vert_variable3_n1, vert_variable4_n1)

                            # if any ([ cond3 <= 1180 for cond3 in Rel_n1]):
                            # # Rel < 1180:
                            #     print ("Liquid laminar")
                            # elif  any ([ cond3 > 1180 for cond3 in Rel_n1]):
                            #     pass

                            # if any ([ cond4 <= 1180 for cond4 in Reg_n1]):
                            #     print ("Gas laminar")
                            # elif  any ([ cond4 > 1180 for cond4 in Reg_n1]):
                            #     pass
                            
                            t = 0
                        else:
                            pass

                        # Time stepping
                        t += dt
                        
                        # Print progress
                        clear_output ()

                        if show_data == 1:
                            print ("Iteration :", int (round (t/dt)), "of", num_steps)
                            print ("Time      :", t, "s")
                                        
                        # Parameters
                        a0 = 3/2
                        a1 = -2
                        a2 = 1/2
                        theta = 1
                        
                        # Amat
                        Am1 = ( (a0*variable1 + a1*variable1_n + a2*variable1_past)/dt*v1 )*dx 
                        Am2 = ( (a0*variable2 + a1*variable2_n + a2*variable1_past)/dt*v2 )*dx 
                        Am3 = ( (a0*variable3 + a1*variable3_n + a2*variable1_past)/dt*v3 )*dx 
                        Am4 = ( (a0*variable4 + a1*variable4_n + a2*variable1_past)/dt*v4 )*dx

                        Am = Am1 + Am2 + Am3 + Am4

                        # Variational form
                        F = visc*Cm - Bm
                        F_n = visc*Cm_n - Bm_n
                        R = Am - theta*F - (1 - theta)*F_n 

                        # Compute directional derivative about u in the direction of du (Jacobian)
                        dF = derivative (R, var, dvar)

                        # Define transient solver function
                        transientsolver = solver_linear (R, variable, bcs, dF)
                        transientsolver.solve ()

                        # Split of the vector var
                        (variable1, variable2, variable3, variable4) = variable.split (deepcopy = True)
                        
                        # L2 norm
                        # infonorm = variable1.vector ().norm("l2")
                        # print ("Solution vector norm (0): {!r}".format (infonorm)) 
                        # L2norm_variable1.append (variable1.vector ().norm ("l2"))
                        # L2norm_variable2.append (variable2.vector ().norm ("l2"))   
                        # L2norm_variable3.append (variable3.vector ().norm ("l2"))   
                        # L2norm_variable4.append (variable4.vector ().norm ("l2"))   

                        # Nodal values (current)
                        n_val1_n1 = np.array(variable1.vector ())
                        n_val2_n1 = np.array(variable2.vector ())
                        n_val3_n1 = np.array(variable3.vector ())
                        n_val4_n1 = np.array(variable4.vector ())

                        nodal_variable1_n1 = n_val1_n1 [::-1]
                        nodal_variable2_n1 = n_val2_n1 [::-1]
                        nodal_variable3_n1 = n_val3_n1 [::-1]
                        nodal_variable4_n1 = n_val4_n1 [::-1]

                        # print ("n+1", nodal_variable1_n1)

                        # Compute vertex values (next step)
                        vert_variable1 = variable1.compute_vertex_values (mesh)
                        vert_variable2 = variable2.compute_vertex_values (mesh)
                        vert_variable3 = variable3.compute_vertex_values (mesh)
                        vert_variable4 = variable4.compute_vertex_values (mesh)

                        vert_variable1_n1 = []
                        vert_variable2_n1 = []
                        vert_variable3_n1 = []
                        vert_variable4_n1 = []

                        for cond_vert in range (len (coordinates)):
                            vert_variable1_n1.append (vert_variable1[cond_vert])
                            vert_variable2_n1.append (vert_variable2[cond_vert])
                            vert_variable3_n1.append (vert_variable3[cond_vert])
                            vert_variable4_n1.append (vert_variable4[cond_vert])
                    
                        vert_variable1_n1 = np.asarray (vert_variable1_n1)
                        vert_variable2_n1 = np.asarray (vert_variable2_n1)
                        vert_variable3_n1 = np.asarray (vert_variable3_n1)
                        vert_variable4_n1 = np.asarray (vert_variable4_n1)

                        if show_data == 1:
                            print ("variable1_n1 = ", vert_variable1_n1)

                        # # Well-posedness check
                        # i = 0
                        # for variable1_test, variable2_test, variable3_test, variable4_test in zip (vert_variable1_n1, vert_variable2_n1, vert_variable3_n1, vert_variable4_n1):
                        #     Aeval = np.asarray (Amat (variable1_test, variable2_test, variable3_test, variable4_test))
                        #     Beval = np.asarray (Bmat (variable1_test, variable2_test, variable3_test, variable4_test))

                        #     m_test, _ = eig (Beval, b = Aeval, overwrite_a = True, overwrite_b = True, check_finite = True)

                        #     listreal = []
                        #     listimag = []

                        #     for cond0 in range (len (m_test)):
                        #         realpart = m_test [cond0].real                
                        #         imagpart = m_test [cond0].imag
                                
                        #         listimag.append (imagpart)
                        #         listreal.append (realpart)
                            
                        #     if any ([ cond1 != 0 for cond1 in listimag ]) :
                        #         print ("INFO: ill-posed equation system")
                        #         break
                        #     else:
                        #         i += 1
                        #     break

                        # Condition for low liquid level of transition to intermittent flow
                        # if any ([cond2 <= 0 for cond2 in vert_variable1_n1]) :
                        #     print ("INFO: gas phase dominant. Low liquid level. Liquid equations vanish")
                        #     break
                        # elif any ([cond2 >= 1 for cond2 in vert_variable1_n1]):
                        #     print ("INFO: liquid phase dominant. Intermittent flow. Gas equations vanish")
                        #     break
                        # else:
                        #     pass
                        # # Condition for positive velocities and pressure
                        # if any ([ cond3 <= 0 for cond3 in vert_variable2_n1]) :
                        #     print ("INFO: negative liquid velocity")
                        #     break
                        # else:
                        #     pass

                        # if any ([ cond4 <= 0 for cond4 in vert_variable3_n1]) :
                        #     print ("INFO: negative gas velocity")
                        #     break
                        # else:
                        #     pass

                        # if any ([ cond5 <= 0 for cond5 in vert_variable4_n1]) :
                        #     print ("INFO: negative pressure")
                        #     break
                        # else:
                        #     pass
                        
                        # # Reynolds check
                        # Rel_n1 = Re_l (vert_variable1_n1, vert_variable2_n1)
                        # Reg_n1 = Re_g (vert_variable1_n1, vert_variable3_n1, vert_variable4_n1)

                        # if any ([ cond3 <= 1180 for cond3 in Rel_n1]):
                        # # Rel < 1180:
                        #     print ("Liquid laminar")
                        # elif  any ([ cond3 > 1180 for cond3 in Rel_n1]):
                        #     pass

                        # if any ([ cond4 <= 1180 for cond4 in Reg_n1]):
                        #     print ("Gas laminar")
                        # elif  any ([ cond4 > 1180 for cond4 in Reg_n1]):
                        #     pass

# ===============================================================
# END OF COMPUTATION FOR ALL TIME METHODS
# ===============================================================
            if time_method == 2:
                var_past.vector ()[:] = var_n.vector () 

            variable_n.vector ()[:] = variable.vector () 

# ===============================================================
# EIGENSPECTRUM
# ===============================================================
            maprealeig = [[ ] for iiiii in range (len (vert_variable1_n1))]
            mapimageig = [[ ] for iiiii in range (len (vert_variable1_n1))]

            list_unstable = []
            
            iiiiii = 0
            for var1_test, var2_test, var3_test, var4_test in zip (vert_variable1_n1, vert_variable2_n1, vert_variable3_n1, vert_variable4_n1):
                Aeval = np.asarray (Amat (var1_test, var2_test, var3_test, var4_test))
                Beval = np.asarray (Bmat (var1_test, var2_test, var3_test, var4_test))
                Ceval = np.asarray (Cmat_lin (var1_test, var2_test, var3_test, var4_test))

                Acomplex   = Aeval.dot (1j)
                Bcomplex   = Beval.dot (1j)
                
                m_eig, _ = eig ( (-wavenumber_fourier*Bcomplex - Ceval), b =  -Acomplex, overwrite_a = True, overwrite_b = True, check_finite = True) 

                listreal_eig = []
                listimag_eig = []

                for cond8 in range (len (m_eig)):
                    realpart_eig = m_eig [cond8].real
                    imagpart_eig = m_eig [cond8].imag
                    
                    listimag_eig.append (imagpart_eig)
                    listreal_eig.append (realpart_eig)
                
                    mapimageig[iiiiii] = np.array(listimag_eig)
                    maprealeig[iiiiii] = np.array(listreal_eig)

                if all ([ cond11 > 0 for cond11 in mapimageig[iiiiii]]):
                    # print("nao imprimir")
                    co = 'stable1'
                    iiiiii += 1
                elif any ([ cond11 < 0 for cond11 in mapimageig[iiiiii]]):
                    # print("Unstable")
                    # print ("este valoooor = ", j_l)
                    co = 'unstable1'
                    j_lmapstability_disc [k, j] = j_l
                    break

            if co == 'unstable1':
                break
            else:
                i += 1
            
            # Delete variables
            del variable1
            del variable2
            del variable3
            del variable4

        j += 1
    k += 1

> Plot 1 figure (discrete VKH limits dor 3 inclination angles)

In [ ]:
x = j_gvector
fig, ax = plt.subplots ()

# Plot discrete VKH limits
for i in range(testpointsbeta):
    ax.loglog (x,
               j_lmapstability_disc [i, :],
               liststyles [i],
               color = listcolor [0] ,
               label = r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' [°]')

# Plot test points
for i in range (testpointsj_l_scatter):
    ax.scatter (j_gscatter [i], 
                j_lscatter [i],
                s = areatestpoints, 
                color = listcolor [0], 
                marker = listmarkers[i], 
                edgecolors = listcolor [0], 
                linewidths = 1, 
                alpha = alphascatter)

    # Set xscale
    ax.set_xscale ('log')

    #ax.scatter (mapreal [number], mapimag [number], s = area, marker = listmarkers[number], color='k', alpha = 0.5, label = '$j_1$' + '=' +str("%.2f" %j1vector [number]) + ' m/s')

# Savitzky-Golay Filter
# for i in range (testpointsbeta): #scipy.signal.savgol_filter: window size 51, polynomial order 3  *******63, 1
#     ax.loglog (x, scipy.signal.savgol_filter(j1mapstability [i,:], 93, 1), liststyles [i], color = 'k', label = '$beta$' + ' = '+str("%.2f" %betavectordeg [i]) + ' deg')

#plt.title('Well-posedness limit for horizontal and inclined pipes')

plt.ylim ((minj_l, maxj_l))
plt.xlim ((minj_g, maxj_g))

#tick_params(labeltop=False, labelright=True)
plt.grid (True, which = "both")
# plt.rcParams ['figure.figsize'] = mapsize
leg = ax.legend (loc = 'best', frameon = True, fontsize = label_size, shadow = True)

matplotlib.rc ('xtick', labelsize = label_size)     
matplotlib.rc ('ytick', labelsize = label_size)

ax.xaxis.set_tick_params (which ='major', direction ='in', top='on')
ax.xaxis.set_tick_params (which ='minor', direction ='in', top='on')
ax.yaxis.set_tick_params (which ='major', direction ='in', right='on')
ax.yaxis.set_tick_params (which ='minor', direction ='in', right='on')

ax.set_xlabel (r'$u_{sg}$ [m/s]', fontsize = label_size)
ax.set_ylabel (r'$u_{sl}$ [m/s]', fontsize = label_size)

# Save figure
fig.set_size_inches (mapsize)
plt.savefig ('results/figures/disc_maps/fig1.pdf', optimize = True, transparent = True, dpi = dpi_elsevier)

plt.show ()

> Plot 3 figures of discrete VKH limits (validation, VKH)

In [ ]:
x = j_gvector

i = 0
for i in range (testpointsbeta):
    fig, ax    = plt.subplots ()
    criteria   = ["Discrete VKH", "Differential IKH"]
    
    # plot differential IKH limits
    ax.loglog (x,
               j_lmapwellposedness [i, :],
               liststyles [i + 1],
               color = listcolor [0],
               label = criteria [1] + ' , ' + r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' $\it{deg}$')

    # plot discrete VKH limits
    ax.loglog (x, 
                j_lmapstability_disc [i, :], 
                liststyles [i], 
                color = listcolor [0], 
                label = criteria [0] + ' , ' + r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' $\it{deg}$')
    
    # Plot validation 
    # Horizontal flow (KH instabilities)
    if all ([D == 0.078, i == 0]):
        x1,y1 = np.loadtxt('validation/Sanderse2017_horizontal/well-posedness_sanderse.csv',
                    unpack = True, 
                    delimiter = ',')
        x2,y2 = np.loadtxt('validation/Sanderse2017_horizontal/globalstability_sanderse.csv',
                        unpack = True, 
                        delimiter = ',')
        
        # fig, ax = plt.subplots()
        ax.loglog(x1,y1, 'ko-', label = 'Differential IKH (Sanderse et al., 2017)')
        ax.loglog(x2,y2, 'kv-', label = 'Discrete VKH (Sanderse et al., 2017)')

        # Stability regions
        # plt.text(0.15, 0.05, 'Well-posed VKH stable')
        # plt.text(0.5, 0.4, 'Well-posed VKH unstable')
        # plt.text(0.9, 3, 'Ill-posed')

    # Horizontal flow (experimental)
    if all ([D == 0.051, i == 0]):
        x1,y1 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_IKH.csv',
                        unpack = True, 
                        delimiter = ',')
        x2,y2 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_VKH.csv',
                        unpack = True, 
                        delimiter = ',')
        # x3,y3 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/shoham1982_0deg.csv',
                        # unpack = True, 
                        # delimiter = ',')
        # x4,y4 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_hlD.csv',
                        # unpack = True, 
                        # delimiter = ',')

        # fig, ax = plt.subplots(1)

        ax.loglog(x1,y1, 'ko-', alpha = alphascatter, label = 'Differential IKH (Barnea and Taitel, 1994)')
        ax.loglog(x2,y2, 'kv-', alpha = alphascatter, label = 'Experimental VKH (Barnea and Taitel, 1994)')
        # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
        # ax.loglog(x3,y3, 'k^-', alpha = alphascatter, label = 'VKH (Shoham, 1982)')
        # ax.loglog(x4,y4, 'ks-', alpha = alphascatter, label = r'$\alpha_l$ = 0.5')

        # Stability regions
        # plt.text(0.15, 0.05, 'Well-posed VKH stable')
        # plt.text(0.5, 0.4, 'Well-posed VKH unstable')
        # plt.text(0.9, 3, 'Ill-posed')

    # Upward flow 0.25 deg (experimental)
    if all ([D == 0.051, i == 1]):
        x1,y1 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_IKH.csv',
                        unpack = True, 
                        delimiter = ',')
        x2,y2 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_VKH.csv',
                        unpack = True, 
                        delimiter = ',')
        # x3,y3 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea 1994_025/shoham1982_025deg.csv',
                        # unpack = True, 
                        # delimiter = ',')
        # x4,y4 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_hlD.csv',
        #                 unpack = True, 
        #                 delimiter = ',')

        # fig, ax = plt.subplots(1)

        ax.loglog(x1,y1, 'ko-', alpha = alphascatter, label = 'Differential IKH (Barnea and Taitel, 1994)')
        ax.loglog(x2,y2, 'kv-', alpha = alphascatter, label = 'Experimental VKH (Barnea and Taitel, 1994)')
        # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
        # ax.loglog(x3,y3, 'k^-', alpha = alphaplot, label = 'VKH (Shoham, 1982)')
        # ax.loglog(x4,y4, 'ks-', alpha = alphaplot, label = r'$\alpha_l$ = 0.5')

        handles, labels = ax.get_legend_handles_labels()

        handles = [handles[0], handles[1], handles[2]]
        labels = [labels[0], labels[1], labels[2]]

        leg1  = ax.legend (handles,
                        labels,
                        shadow = True, 
                        frameon = True,
                        fontsize = label_size)

        # Stability regions
        # plt.text(0.3, 0.05, 'Well-posed VKH stable')
        # plt.text(5.0, 0.02, 'Well-posed VKH unstable')
        # plt.text(1.1, 2, 'Ill-posed')

    # Downward flow -5 deg (experimental)
    if all ([D == 0.051, i == 2]):
        x1,y1 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994-5/barnea1994_-5deg_IKH.csv',
                        unpack = True, 
                        delimiter = ',')
        # x2,y2 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994-5/shoham1982_-5deg.csv',
        #                 unpack = True, 
        #                 delimiter = ',')
        # x3,y3 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994-5/barnea1994_-5deg_hlD.csv',
                        # unpack = True, 
                        # delimiter = ',')

        # fig, ax = plt.subplots(1)
        ax.loglog(x1,y1, 'ko-', alpha = alphascatter, label = 'Differential IKH (Barnea and Taitel, 1994)')
        # ax.loglog(x2,y2, 'kv-', alpha = alphascatter, label = 'Experimental IKH (Barnea and Taitel, 1994)')
        # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
        # ax.loglog(x2,y2, 'k^-', alpha = alphascatter, label = 'Experimental VKH (Shoham, 1982)')
        # ax.loglog(x3,y3, 'ks-', alpha = alphaplot, label = r'$\alpha_l$ = 0.5')

        handles, labels = ax.get_legend_handles_labels()

        handles = [handles[0], handles[1], handles[2]]
        labels = [labels[0], labels[1], labels[2]]

        leg1  = ax.legend (handles,
                        labels,
                        shadow = True, 
                        frameon = True,
                        fontsize = label_size)

        # Stability regions
        # plt.text(3.0, 0.012, 'Well-posed VKH stable')
        # plt.text(0.3, 0.5, 'Well-posed VKH unstable')
        # plt.text(1.5, 3, 'Ill-posed')

    # Savitzky-Golay Filter ****83, 1
    # ax.loglog (x,
    #            scipy.signal.savgol_filter(j_lmapstability [i,:], 93, 1), #93, 1
    #            liststyles [0], color = listcolor [0],
    #            label = criteria [0] + ' , ' + '$β$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' [°]')
    
    # # line of constant alpha
    # ax.loglog (x,
    #            j_llinemap [i, :],
    #            liststyles [1],
    #            color = listcolor [0],
    #            label =  r'$\alpha_l$ = 0.5 [-]')
  # Fill
    # ax.fill_between(x,
    #                 scipy.signal.savgol_filter(j_lmapstability [i,:], 9, 1),
    #                 j_lmapwellposedness [i, :],
    #                 where = j_lmapwellposedness [i, :] >= scipy.signal.savgol_filter(j_lmapstability [i,:], 113, 1),
    #                 facecolor = 'silver',
    #                 interpolate = True)    
    #ax.scatter (mapreal [number], mapimag [number], s = area, marker = listmarkers[number], color='k', alpha = 0.5, label = '$j_1$' + '=' +str("%.2f" %j1vector [number]) + ' m/s') 
    
    #Scatter
#     ax.plot(j_gtest2 [i, :], j_ltest2 [i, :], 's', color = 'black')


    # Plot test points
    for ii in range (testpointsj_l_scatter):
        ax.scatter (j_gscatter [ii], j_lscatter [ii], s = areatestpoints, color = listcolor [4], marker = listmarkers[i], edgecolors = listcolor [0], linewidths = 1.5, alpha = 0.5)
        # ax.set_xscale ('symlog')

    ax.set_xscale ('log')
    
    plt.ylim ((minj_l, maxj_l))
    plt.xlim ((minj_g, maxj_g))

    #plt.title('Well-posedness limit for horizontal and inclined pipes')
    #tick_params(labeltop=False, labelright=True)
    plt.grid (True, which = "both")

    leg = ax.legend (loc = 'best', frameon = True, fontsize = label_size, shadow = True)

    # x and y ticks
    matplotlib.rc ('xtick', labelsize = label_size)     
    matplotlib.rc ('ytick', labelsize = label_size)

    # Ticks
    ax.xaxis.set_tick_params (which = 'major', direction ='in', top ='on')
    ax.xaxis.set_tick_params (which = 'minor', direction ='in', top ='on')
    ax.yaxis.set_tick_params (which = 'major', direction ='in', right ='on')
    ax.yaxis.set_tick_params (which = 'minor', direction ='in', right ='on')

    ax.set_xlabel(r'$u_{sg}$ $[\it{m \; s^{-1}}]$', fontsize = label_size)
    ax.set_ylabel(r'$u_{sl}$ $[\it{m \; s^{-1}}]$', fontsize = label_size)
    
    # Save plot
    fig.set_size_inches (mapsize)
    plt.savefig('results/figures/disc_maps/fig2_'+ str(i) +'.pdf', optimize = True, transparent = True, dpi = dpi_elsevier)

    # Show plot
    plt.show ()

    i += 1